# Predicciones.
#### En este documento se hacen predicciones de la intensidad y velocidad media del mejor modelo entrenado en la carpeta `models` basadas en datos en tiempo real.

El primer paso es importar las librerías necesarias para el correcto funcionamiento del cuaderno de Jupyter.

In [ ]:
# Librerías estándar
import pickle  # Serialización de objetos Python
import xml.etree.ElementTree as ET  # Lectura y procesamiento de archivos XML

# Manejo de datos
import numpy as np  # Operaciones numéricas con arrays
import pandas as pd  # Manipulación de datos tabulares

# Modelado y persistencia
from keras.models import load_model  # Carga de modelos Keras preentrenados
import joblib  # Serialización de modelos y objetos (más eficiente que pickle para modelos grandes)

Ahora ya se inicia la simulación para calcular la distancia y tiempo para cada paso del camino mínimo. Se calculará el tiempo para cada movimiento del trayecto mediante la fórmula t = distancia / velocidad.

En este primer caso se realiza la simulación de que se calcula una hora de llegada a partir de una de salida.

Luego se carga el documento XML en tiempo real.

In [ ]:
tree = ET.parse('../data/pm.xml')
root = tree.getroot()

Ahora, se recopila únicamente la información que nos interesa, que se trata de la información de un subconjunto de todos los datos. Mientras que se recopila, se transforman las columnas para poder meter directamente la información en el modelo y evitar hacer transformaciones de nuevo.

In [ ]:
predicciones_intensidad = {}
predicciones_vmed = {}

for element in ['6798', '6767', '4486', '4362', '3745', '4376', '4373', '5768', '6800', '3715', '3499', '5775', '9907', '9926', '5412', '3498']:
    datos = []
    fecha = root.find('.//fecha_hora').text
    for pm in root.findall('.//pm'):
        idelem = pm.find('idelem')
        if idelem.text == element:
            ocupacion = float(pm.find('ocupacion').text)
            carga = float(pm.find('carga').text)
            error = pm.find('error').text

            datos.append({
                'ocupacion': ocupacion,
                'carga': carga,
                'error': error,
                'periodo_integracion': 15,
                'fecha': fecha,
                'intensidad': 0,
                'vmed': 0
            })
            
    df = pd.DataFrame(datos)
    df['fecha'] = pd.to_datetime(df['fecha'])

    df['hora'] = df['fecha'].dt.hour
    df['dow'] = df['fecha'].dt.dayofweek  # Lunes=0
    df['doy'] = df['fecha'].dt.dayofyear
    df['month'] = df['fecha'].dt.month

    df['hora_sin'] = np.sin(2 * np.pi * df['hora'] / 24)
    df['hora_cos'] = np.cos(2 * np.pi * df['hora'] / 24)

    df['dow_sin'] = np.sin(2 * np.pi * df['dow'] / 7)
    df['dow_cos'] = np.cos(2 * np.pi * df['dow'] / 7)

    df['doy_sin'] = np.sin(2 * np.pi * df['doy'] / 365)
    df['doy_cos'] = np.cos(2 * np.pi * df['doy'] / 365)

    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

    df['error'] = (df['error'] != 'N').astype(int)

    columnas_modelo = [
        'ocupacion', 'carga', 'error', 'periodo_integracion',
        'hora_sin', 'hora_cos', 'dow_sin', 'dow_cos',
        'doy_sin', 'doy_cos', 'month_sin', 'month_cos', 'intensidad', 'vmed'
    ]

    X = df[columnas_modelo].values

    # Cargar el modelo
    modelo = load_model(f"../models/final/{element}.h5")

    # Cargar el scaler
    scaler = joblib.load(f"../models/final/{element}.pkl")  # o con pickle.load(open(...))

    print('Modelo y scaler cargados correctamente.')

    # Escalar como siempre
    X_scaled = scaler.transform(X)

    # El modelo solo necesita las primeras 12 columnas
    X_scaled_model_input = X_scaled[:, :12]

    # Reshape para LSTM
    X_lstm = X_scaled_model_input.reshape((X_scaled_model_input.shape[0], 1, X_scaled_model_input.shape[1]))

    # Predecir
    pred = modelo.predict(X_lstm)

    # Crear un array lleno de ceros (mismo número de columnas que usó el scaler)
    salida_dummy = np.zeros((pred.shape[0], scaler.scale_.shape[0]))

    # Insertar tus predicciones en las posiciones correctas (por ejemplo, últimas 2 columnas)
    # Suponiendo que intensidad y vmed eran las últimas 2 columnas (índices 12 y 13):
    salida_dummy[:, 12:] = pred

    # Desescalar todo
    salida_original = scaler.inverse_transform(salida_dummy)

    # Extraer solo la parte que nos interesa
    intensidad_pred, vmed_pred = salida_original[:, 12], salida_original[:, 13]

    predicciones_intensidad[element] = intensidad_pred[0]
    predicciones_vmed[element] = round(vmed_pred[0], 2)

Las velocidades rpedichas valen la mayoría 0 ya que esta variablestá disponible únicamente para elementos interurbanos, y en este caso la mayoría de los IDS sin urbanos. Para estimar la velocidad, se hará proporcionalmente a las intensidades.

In [ ]:
def corregir_velocidades(velocidades, intensidades, min_vel=5, max_vel=30):
    '''
    Corrige las velocidades nulas asignando proporcionalmente a las intensidades.
    '''
    # IDs con velocidad 0 o muy baja (por seguridad usamos un umbral pequeño, por si hay floats tipo 0.0)
    ids_corregir = [id_ for id_, v in velocidades.items() if np.isclose(v, 0.0)]

    # Obtener intensidades correspondientes a esos IDs
    intensidades_corregir = {id_: intensidades[id_] for id_ in ids_corregir}

    # Extraer valores de intensidad
    valores_intensidad = np.array(list(intensidades_corregir.values()), dtype=float)

    min_int = valores_intensidad.min()
    max_int = valores_intensidad.max()

    # Evitar división por cero (todos tienen misma intensidad)
    if np.isclose(min_int, max_int):
        velocidad_media = (min_vel + max_vel) / 2
        for id_ in ids_corregir:
            velocidades[id_] = np.float64(velocidad_media)
        return velocidades

    # Asignar velocidades proporcionalmente e inversamente
    for id_, intensidad in intensidades_corregir.items():
        intensidad_norm = (intensidad - min_int) / (max_int - min_int)  # escalar entre 0 y 1
        velocidad = max_vel - intensidad_norm * (max_vel - min_vel)
        velocidades[id_] = np.float64(round(velocidad, 2))

    return velocidades

predicciones_vmed = corregir_velocidades(predicciones_vmed, predicciones_intensidad)

### Guardamos las predicciones

Se guardan las predicciones para poder usarlas para calcular luego el camino mínimo y el tiempo de desplazamiento. También se guarda la fecha de los datos en tiempo real con el fin de obtener automáticamente la hora desde la que basar las simulaciones.

In [ ]:
print(predicciones_intensidad)

with open('./predicciones_intensidad.pkl', 'wb') as f:
    pickle.dump(predicciones_intensidad, f)


In [ ]:
print(predicciones_vmed)

with open('./predicciones_vmed.pkl', 'wb') as f:
    pickle.dump(predicciones_vmed, f)

In [ ]:
# Guardamos la fecha para sumar el tiempo que se realiza en el trayecto del camino mínimo

with open('./fecha.pkl', 'wb') as f:
    pickle.dump(fecha, f)